**Importing required libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

import warnings
warnings.filterwarnings('ignore')

Reading data

In [ ]:
train=pd.read_csv('../input/bike-sharing-demand/train.csv', parse_dates = ['datetime'])
test=pd.read_csv('../input/bike-sharing-demand/test.csv', parse_dates = ['datetime'])
sampsub = pd.read_csv("../input/bike-sharing-demand/sampleSubmission.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
figure(figsize=(12, 8))
sns.heatmap(train.corr())

**Taking date as a fratures. Converted date columns to year, month, day, hour and weekday column**

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['weekday'] = train['datetime'].dt.weekday

Removing date column

In [ ]:
train = train.drop('datetime', axis = 1)

Applying same conversion on test data also

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['weekday'] = test['datetime'].dt.weekday

In [ ]:
test = test.drop('datetime', axis = 1)

Here temp, atemp, humidity and windspeed features available are not in binary form so using Normalization method for this four column

In [ ]:
col = ['temp', 'atemp', 'humidity', 'windspeed']

In [ ]:
for i in col:
    scaler = StandardScaler()
    train[col]=scaler.fit_transform(train[col])
    test[col]=scaler.transform(test[col])   

In [ ]:
train['count'].hist()

Splitting data into X_train and y_train datasets

In [ ]:
X_train = train.drop(['count', 'casual','registered'], axis = 1)
y_train = train['count']

In y_train data is in skewed form so using log transform to manage data

In [ ]:
y_train = np.log(y_train.values)

checking final shape of train and test data

In [ ]:
X_train.shape, y_train.shape, test.shape

Created random forest algorithm for predictions

In [ ]:
model_rf = RandomForestRegressor(random_state=2021)

applying bagging regression concept for random forest algorithm

In [ ]:
model_bg = BaggingRegressor(base_estimator = model_rf, random_state=2021,oob_score=True, max_features=X_train.shape[1], n_estimators = 15,
                             max_samples=X_train.shape[0])

fitting this model with random forest and bagging regression concept

In [ ]:
model_bg.fit( X_train , y_train )

Predicting results from above model

In [ ]:
y_pred = model_bg.predict(test)

Taking anti log transform of predicted result

In [ ]:
y_pred =np.exp(y_pred)

Creating submission file

In [ ]:
submission = pd.DataFrame({ "datetime": sampsub.datetime, "count": y_pred})

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission_random_forest_bagging.csv", index=False)